# Question:1
1. Find the Top 5 movies (by rating) in every genre

In [4]:
#Package Imports

import pandas as pd
from pandasql import *
pysqldf = lambda q: sqldf(q, globals())

In [5]:
#Create Dataframe from u.data file
#Renaming item_id column in u.data to movie_id 

column_names = ['user_id','movie_id','rating','timestamp' ]
movie_ratings = pd.read_csv('data/u.data',delimiter='\t',names=column_names)
movie_ratings.head(5)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
#Create Dataframe from u.item file

column_names= ['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movie_data =  pd.read_csv('data/u.item',encoding="437",delimiter='|',names=column_names)
movie_data.head(5)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
#Create Dataframe from u.genre file
genre = pd.read_csv('data/u.genre',delimiter='|',names=['genere','id'])
genre.head(5)


,genere,id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [8]:
#Create Dataframe from u.user file
column_names = ['user_id','age','gender','occupation','zip_code']
user_data = pd.read_csv('data/u.user',delimiter='|',names=column_names)
user_data.head(5)

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [9]:
#Inner Join movie_data and movie_ratings based on movie_id  
dataset = movie_ratings.merge(movie_data,on='movie_id',how="inner")
dataset.head(3)

,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,63,242,3,875747190,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
2,226,242,5,883888671,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# For genre Action
query="SELECT count(user_id) as counts,movie_title FROM (select * from dataset where Action=1 and rating = 5) group by movie_id order by counts desc limit 5"
pysqldf(query)

,counts,movie_title
0,325,Star Wars (1977)
1,214,"Godfather, The (1972)"
2,202,Raiders of the Lost Ark (1981)
3,179,Titanic (1997)
4,172,"Empire Strikes Back, The (1980)"


In [11]:
# Gives same result as above
df = dataset[(dataset["Action"] == 1) & (dataset["rating"] == 5)]
query = "SELECT count(user_id) as counts,movie_title FROM df group by movie_id order by counts desc limit 5"
pysqldf(query)

,counts,movie_title
0,325,Star Wars (1977)
1,214,"Godfather, The (1972)"
2,202,Raiders of the Lost Ark (1981)
3,179,Titanic (1997)
4,172,"Empire Strikes Back, The (1980)"


In [13]:
# Snippet to get Top 5 Movies of all genres and saves to csv

genere = pd.read_csv('data/u.genre',delimiter='|',names=['genere','id'])
with open('FirstQuesAnswer.csv', 'a') as f:
    for i in genere['genere']:
        df=dataset[(dataset[str(i)] == 1) & (dataset["rating"] == 5)]
        query = "SELECT count(user_id) as counts,movie_title as 'Top5Moives:' FROM df group by movie_id order by counts desc limit 5"
        df=pysqldf(query)
        data=df.drop("counts",axis=1)
        f.write("-----------------------------------------\n")
        f.write("GenereName: "+i+"\n")
        data.to_csv(f, index=False)